In [1]:
import numpy as np
import matplotlib.pyplot as plt

from preprocessing.preprocessor import Preprocess     #  From usatellite
from models.unet2d.unet2d_model import Unet2d         #  From usatellite
from plot_utils.plotting_utils import SatPlotter      #  From usatellite
from loss.loss_metrics import f1
from loss.loss_metrics import categorical_focal_loss
from loss.loss_metrics import weighted_categorical_crossentropy

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from skimage.transform import resize

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

#TEMP
import tensorflow as tf

ModuleNotFoundError: No module named 'loss.loss_metrics.f1'; 'loss.loss_metrics' is not a package

In [ ]:
aug_imgs = np.load(r'C:\\Users\\Cody\\Documents\\usatellite_data\\aug_imgs.npy')
aug_labels = np.load(r'C:\\Users\\Cody\\Documents\\usatellite_data\\aug_labels.npy')

In [ ]:
preproc = Preprocess()

In [ ]:
imgs_array_scaled = preproc.unit_normalize_dims(aug_imgs)
labels_array_scaled = preproc.unit_normalize_dims(aug_labels)

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(imgs_array_scaled, labels_array_scaled, test_size=0.30, shuffle=True, random_state=42)

In [ ]:
unet2dmodel = Unet2d(input_img=imgs_array_scaled)

In [ ]:
unet = unet2dmodel.get_unet()

In [ ]:
unet.compile(optimizer=Adam(), loss=categorical_focal_loss(n_classes=6, alpha_val=0.25), metrics=["categorical_accuracy", f1])

In [ ]:
results = unet.fit(x_train, y_train, batch_size=2, epochs=10, validation_data=(x_val, y_val))

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["f1"], label="f1")
plt.plot(results.history["val_f1"], label="val_f1")
#plt.plot( np.argmin(results.history["val_f1"]), np.min(results.history["val_f1"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
test_imgs = unet.predict(x_val)

In [ ]:
test_classes = np.argmax(test_imgs, -1)

In [ ]:
satplot = SatPlotter()

In [ ]:
satplot.plot_image_and_result(x_val, y_val, test_classes)

In [ ]:
plt.imshow(test_imgs[10,:,:,0])

In [ ]:
test = np.ones((6))

In [ ]:
test*0.25